In [ ]:
# Colab 환경에서는 필요한 라이브러리들을 먼저 설치합니다.
!pip install konlpy gensim pyLDAvis

# 구글 드라이브에 파일이 있다면 드라이브를 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from konlpy.tag import Okt
from gensim import corpora, models  # gensim 모듈 임포트
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

# 파일 경로를 구글 드라이브 내의 경로로 수정합니다.
excel_file = "/content/금융 접근성_정리된.xlsx"  # 경로 수정
df = pd.read_excel(excel_file)

# LDA 파라미터 설정
NUM_TOPICS = 5
PASSES = 10
RANDOM_STATE = 42

# 형태소 분석기 준비 (Okt 사용)
okt = Okt()

# '본문' 컬럼에서 명사만 추출 (한 글자 단어 제외)
df['tokens'] = df['본문'].apply(lambda x: [token for token in okt.nouns(x) if len(token) > 1] if isinstance(x, str) else [])

# Gensim Dictionary & Corpus 생성
dictionary = corpora.Dictionary(df['tokens'])
# 너무 드물거나, 너무 많이 등장하는 단어 제거 (상황에 맞게 조절)
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# LDA 모델 학습
lda_model = models.LdaModel(corpus=corpus,
                            id2word=dictionary,
                            num_topics=NUM_TOPICS,
                            passes=PASSES,
                            random_state=RANDOM_STATE)

# LDA 토픽 결과 출력
print("LDA 토픽 결과:")
topics = lda_model.print_topics(num_topics=NUM_TOPICS, num_words=10)
for topic in topics:
    print(topic)

# 각 문서의 주도 토픽(dominant topic) 할당 함수
def get_dominant_topic(bow):
    topic_probs = lda_model.get_document_topics(bow)
    if topic_probs:
        return max(topic_probs, key=lambda x: x[1])[0]
    else:
        return None

df['dominant_topic'] = [get_dominant_topic(bow) for bow in corpus]

# 날짜별 토픽 분포 시각화 (문서 수 기준)
topic_time = df.groupby('일자')['dominant_topic'].value_counts().unstack(fill_value=0)
topic_time = topic_time.sort_index()

plt.figure(figsize=(12, 6))
topic_time.plot(marker='o')
plt.title("날짜별 토픽 분포")
plt.xlabel("날짜")
plt.ylabel("문서 수")
plt.legend(title="토픽 번호")
plt.tight_layout()
plt.show()

# pyLDAvis로 LDA 결과 시각화 및 HTML 파일로 저장
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
html_vis_filename = "/content/LDA_접근성vis.html"  # 경로 수정
pyLDAvis.save_html(vis, html_vis_filename)
print(f"LDA 시각화 파일_접근성 저장 완료: {html_vis_filename}")

# 최종 결과를 엑셀 파일로 저장 (dominant topic 컬럼 포함)
output_filename = "/content/LDA_접근성결과.xlsx"  # 경로 수정
df.to_excel(output_filename, index=False)
print(f"LDA 결과 엑셀 파일 저장 완료: {output_filename}")
